In [1]:
from mayavi import mlab
import pandas as pd
import numpy as np
import trimesh as tr
import time

In [2]:
#import the cell reconstruction
file= pd.read_csv('cell_reconstruction1.txt',sep=" ", header=None)

In [3]:
#read the data for the analysis
vertss = []
vert1, vert2 = [x for _, x in file.groupby((file[0] == 'v'))]
vertss.append(vert2)

In [4]:
del vertss[0][0]

In [5]:
vertsss= vertss[0].to_numpy()
verts= vertsss.astype('int')

In [6]:
face = []
face1, face2 = [x for _, x in file.groupby((file[0] == 'f'))]
face.append(face2)

In [7]:
del face[0][0]

In [8]:
facee = face[0].to_numpy()
faces = facee.astype('int')

In [9]:
normal = []
norm1, norm2 = [x for _, x in file.groupby((file[0] == 'vn'))]
normal.append(norm2)

In [10]:
del normal[0][0]

In [11]:
crosses = normal[0].to_numpy()
cross = crosses.astype('int')

In [15]:
#Graphical representation, run if you want to see the cell
x= [vert[0] for vert in verts]
y= [vert[1] for vert in verts]
z= [vert[2] for vert in verts]

mlab.triangular_mesh(x,y,z, faces, representation = 'wireframe')
mlab.show()

In [20]:
def SurfaceOrientationTensor(vertices, faces):
    facesNo= faces.shape[0]
    mesh = tr.Trimesh(vertices=verts, faces=faces)
    Area = mesh.area_faces
    face_normals = mesh.face_normals
    f = np.zeros((3,3))
    
    for k in range(0,facesNo):
        f = f+ Area[k]*np.tensordot(face_normals[k,:],face_normals[k,:],axes=0) #outer product
    f = f/np.sum(Area) #normalise to the total surface area
    return f, face_normals

def principalSOT(f):
    eigenVal, vectors = np.linalg.eig(f) #eigen analysis
    #eigenValReal = eigenVal.real # discard complex part (if any)
    eigenValues = np.sort(eigenVal)[::-1] #sort eigenvalues in descending order

    f1 = eigenValues[0] #Largerst eigenvalue
    f2 = eigenValues[1] #Intermediate eigenvalue
    f3 = eigenValues[2] #Smallest eigenvalue

    C = f3/f1 #Compactness
    F = (f1-f3)/f1 #Flakiness
    E = (f2-f3)/f1 #Elongation
    return C, F, E

In [21]:
start_time = time.time()
# calculate surface orientation tensor
S, face_normals = SurfaceOrientationTensor(verts, faces)

# calculate shape descriptors
Compactness, Flakiness, Elongation = principalSOT(S)

# print results
print("Surface Tensor:", S)
print("Compactness:", Compactness)
print("Flakiness:", Flakiness)
print("Elongation:", Elongation)
print("--- %s seconds ---" % (time.time() - start_time))

Surface Tensor: [[0.34517485 0.04577559 0.04750732]
 [0.04577559 0.3320896  0.0406698 ]
 [0.04750732 0.0406698  0.32273555]]
Compactness: 0.6702999616514728
Flakiness: 0.3297000383485273
Elongation: 0.019930594430729504
--- 0.5006711483001709 seconds ---
